In [51]:
from ax import (
    ComparisonOp,
    Experiment,
    Metric,
    Objective,
    OptimizationConfig,
    OutcomeConstraint,
    ParameterType,
    RangeParameter,
    SearchSpace,
)
from ax.utils.notebook.plotting import render


# ---------------------------------- create search space ----------------------------------

from ax.metrics.l2norm import L2NormMetric
from ax.metrics.hartmann6 import Hartmann6Metric


hartmann_search_space = SearchSpace(
    parameters=[
        RangeParameter(
            name=f"x{i}",
            parameter_type=ParameterType.FLOAT,
            lower=0.0,
            upper=1.0,
        )
        for i in range(6)
    ]
)

In [52]:
# ---------------------------------- create optimization config ----------------------------------
param_names = [f"x{i}" for i in range(6)]
optimization_config = OptimizationConfig(
    objective=Objective(
        metric=Hartmann6Metric(name="hartman6", param_names=param_names),
        minimize=True,
    ),
    outcome_constraints=[
        OutcomeConstraint(
            metric=L2NormMetric(
                name="l2norm", param_names=param_names, noise_sd=0.2
            ),
            op=ComparisonOp.LEQ,
            bound=1.25,
            relative=False,
        )
    ],
)

In [53]:
# ---------------------------------- define a runner ----------------------------------
from ax import Runner


class MyRunner(Runner):
    def run(self, trial):
        trial_metadata = {"name": str(trial.index)}
        return trial_metadata

In [54]:
# ---------------------------------- create experiment ----------------------------------
exp = Experiment(
    name="test_hartmann",
    search_space=hartmann_search_space,
    optimization_config=optimization_config,
    runner=MyRunner(),
)

In [55]:
exp.fetch_data()

[INFO 03-09 12:29:34] ax.core.experiment: No trials are in a state expecting data.


Data(df=
| metric_name   | arm_name   | mean   | sem   |
|---------------|------------|--------|-------|)

In [56]:
# ---------------------------------- Perform Optimization ----------------------------------
from ax.modelbridge.registry import Models

NUM_SOBOL_TRIALS = 5
NUM_BOTORCH_TRIALS = 15

print(f"Running Sobol initialization trials...")
sobol = Models.SOBOL(search_space=exp.search_space)

for i in range(NUM_SOBOL_TRIALS):
    # Produce a GeneratorRun from the model, which contains proposed arm(s) and other metadata
    generator_run = sobol.gen(n=1)
    # Add generator run to a trial to make it part of the experiment and evaluate arm(s) in it
    trial = exp.new_trial(generator_run=generator_run)
    # Start trial run to evaluate arm(s) in the trial
    trial.run()
    # Mark trial as completed to record when a trial run is completed
    # and enable fetching of data for metrics on the experiment
    # (by default, trials must be completed before metrics can fetch their data,
    # unless a metric is explicitly configured otherwise)
    trial.mark_completed()

for i in range(NUM_BOTORCH_TRIALS):
    print(
        f"Running BO trial {i + NUM_SOBOL_TRIALS + 1}/{NUM_SOBOL_TRIALS + NUM_BOTORCH_TRIALS}..."
    )
    # Reinitialize GP+EI model at each step with updated data.
    gpei = Models.BOTORCH_MODULAR(experiment=exp, data=exp.fetch_data())
    generator_run = gpei.gen(n=1)
    trial = exp.new_trial(generator_run=generator_run)
    trial.run()
    trial.mark_completed()

Running Sobol initialization trials...
Running BO trial 6/20...
Running BO trial 7/20...
Running BO trial 8/20...
Running BO trial 9/20...
Running BO trial 10/20...
Running BO trial 11/20...
Running BO trial 12/20...
Running BO trial 13/20...
Running BO trial 14/20...
Running BO trial 15/20...
Running BO trial 16/20...
Running BO trial 17/20...
Running BO trial 18/20...
Running BO trial 19/20...
Running BO trial 20/20...


In [57]:
trial_data = exp.fetch_trials_data([NUM_SOBOL_TRIALS + NUM_BOTORCH_TRIALS - 1])
trial_data.df

,arm_name,metric_name,mean,sem,trial_index,n,frac_nonnull
0,19_0,hartman6,-0.719056,0.0,19,10000,-0.719056
1,19_0,l2norm,1.306705,0.2,19,10000,1.306705


In [58]:
trial_data = exp.fetch_data()
trial_data.df

,arm_name,metric_name,mean,sem,trial_index,n,frac_nonnull
0,0_0,hartman6,-0.023486,0.0,0,10000,-0.023486
1,0_0,l2norm,1.860939,0.2,0,10000,1.860939
2,1_0,hartman6,-0.174101,0.0,1,10000,-0.174101
3,1_0,l2norm,1.148897,0.2,1,10000,1.148897
4,2_0,hartman6,-0.114524,0.0,2,10000,-0.114524
5,2_0,l2norm,1.542466,0.2,2,10000,1.542466
6,3_0,hartman6,-0.440006,0.0,3,10000,-0.440006
7,3_0,l2norm,0.993774,0.2,3,10000,0.993774
8,4_0,hartman6,-0.097379,0.0,4,10000,-0.097379
9,4_0,l2norm,1.344947,0.2,4,10000,1.344947


In [59]:
import numpy as np

objective_means = np.array(
    [[trial.objective_mean for trial in exp.trials.values()]]
)
objective_means

array([[-0.02348647, -0.17410069, -0.11452382, -0.44000649, -0.09737855,
        -0.01233362, -0.31073767, -0.15565883, -0.25270467, -0.22341224,
        -0.40865718, -0.52991743, -0.63685782, -0.69650389, -0.0922461 ,
        -0.62062015, -0.90402158, -1.16526805, -1.45287593, -0.71905565]])

In [60]:
from src.utils.metric import extract_metric

results = extract_metric(exp=exp, metric_name="hartman6")

In [61]:
results

array([-0.02348647, -0.17410069, -0.11452382, -0.44000649, -0.09737855,
       -0.01233362, -0.31073767, -0.15565883, -0.25270467, -0.22341224,
       -0.40865718, -0.52991743, -0.63685782, -0.69650389, -0.0922461 ,
       -0.62062015, -0.90402158, -1.16526805, -1.45287593, -0.71905565])

In [62]:
# ---------------------------------- plot results ----------------------------------

import numpy as np
from ax.plot.trace import optimization_trace_single_method

objective_means = np.array(
    [[trial.objective_mean for trial in exp.trials.values()]]
)
best_objective_plot = optimization_trace_single_method(
    y=np.minimum.accumulate(objective_means, axis=1),
    optimum=-3.32237,  # Known minimum objective for Hartmann6 function.
)
render(best_objective_plot)

In [63]:
# ---------------------------------- Defining custom metrics ----------------------------------
from ax import Data
import pandas as pd


class BoothMetric(Metric):
    def fetch_trial_data(self, trial):
        records = []
        for arm_name, arm in trial.arms_by_name.items():
            params = arm.parameters
            records.append(
                {
                    "arm_name": arm_name,
                    "metric_name": self.name,
                    "trial_index": trial.index,
                    # in practice, the mean and sem will be looked up based on trial metadata
                    # but for this tutorial we will calculate them
                    "mean": (params["x1"] + 2 * params["x2"] - 7) ** 2
                    + (2 * params["x1"] + params["x2"] - 5) ** 2,
                    "sem": 0.0,
                }
            )
        return Data(df=pd.DataFrame.from_records(records))

    def is_available_while_running(self) -> bool:
        return True

In [64]:
# ---------------------------------- save to json or sql ----------------------------------
from ax.storage.registry_bundle import RegistryBundle

bundle = RegistryBundle(
    metric_clss={BoothMetric: None, L2NormMetric: None, Hartmann6Metric: None},
    runner_clss={MyRunner: None},
)

from ax.storage.json_store.load import load_experiment
from ax.storage.json_store.save import save_experiment

save_experiment(
    exp, "experiment.json", encoder_registry=bundle.encoder_registry
)

In [65]:
loaded_experiment = load_experiment(
    "experiment.json", decoder_registry=bundle.decoder_registry
)

loaded_experiment

Experiment(test_hartmann)

In [66]:
import numpy as np

objective_means = np.array(
    [[trial.objective_mean for trial in exp.trials.values()]]
)

In [67]:
exp.fetch_data().df

,arm_name,metric_name,mean,sem,trial_index,n,frac_nonnull
0,0_0,hartman6,-0.023486,0.0,0,10000,-0.023486
1,0_0,l2norm,2.016825,0.2,0,10000,2.016825
2,1_0,hartman6,-0.174101,0.0,1,10000,-0.174101
3,1_0,l2norm,1.652139,0.2,1,10000,1.652139
4,2_0,hartman6,-0.114524,0.0,2,10000,-0.114524
5,2_0,l2norm,1.057455,0.2,2,10000,1.057455
6,3_0,hartman6,-0.440006,0.0,3,10000,-0.440006
7,3_0,l2norm,1.259232,0.2,3,10000,1.259232
8,4_0,hartman6,-0.097379,0.0,4,10000,-0.097379
9,4_0,l2norm,1.323519,0.2,4,10000,1.323519


In [70]:
best_objective_plot = optimization_trace_single_method(
    y=np.minimum.accumulate(objective_means, axis=1),
    optimum=-3.32237,  # Known minimum objective for Hartmann6 function.
)
render(best_objective_plot)